In [1]:
import polars as pl
import io

# Replace 'your_file.tsv' with the path to your TSV file
df = pl.read_csv('participants.tsv', separator='\t', null_values=["n/a"], infer_schema_length=200)

print(df)

shape: (192, 87)
┌────────────┬────────────┬────────────┬───────────┬───┬───────────┬───────────┬───────────┬───────┐
│ participan ┆ second_pha ┆ session_or ┆ APOE_rs42 ┆ … ┆ non-HDL_c ┆ LDL_chole ┆ triglycer ┆ HSV_r │
│ t_id       ┆ se         ┆ der        ┆ 9358      ┆   ┆ holestero ┆ sterol    ┆ ides      ┆ ---   │
│ ---        ┆ ---        ┆ ---        ┆ ---       ┆   ┆ l         ┆ ---       ┆ ---       ┆ i64   │
│ str        ┆ i64        ┆ i64        ┆ str       ┆   ┆ ---       ┆ f64       ┆ f64       ┆       │
│            ┆            ┆            ┆           ┆   ┆ f64       ┆           ┆           ┆       │
╞════════════╪════════════╪════════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════╡
│ sub-01     ┆ 1          ┆ 1          ┆ T/T       ┆ … ┆ 136.4     ┆ 100.48    ┆ 179.6     ┆ 1     │
│ sub-02     ┆ 1          ┆ 1          ┆ T/T       ┆ … ┆ 117.3     ┆ 84.56     ┆ 163.7     ┆ 1     │
│ sub-03     ┆ 1          ┆ 0          ┆ T/T       ┆ … ┆ 109.6     ┆ 100.8

In [17]:
# Count the different levels of APOE_haplotype
for col in  ["APOE_haplotype", "APOE_rs429358", "education"]:
    apoe_counts = df[col].value_counts()

    print(f"{col} Counts:")
    display(apoe_counts)

APOE_haplotype Counts:


APOE_haplotype,count
str,u32
"""e4/e4""",2
"""e2/e2""",1
"""e3/e2 """,2
"""e3/e3""",119
"""e2/e4""",3
"""e3/e4""",46
"""e3/e2""",19


APOE_rs429358 Counts:


APOE_rs429358,count
str,u32
"""T/?""",1
"""T/T""",140
"""T/C""",49
"""C/C""",2


education Counts:


education,count
i64,u32
null,18
1,20
3,149
2,5


In [18]:
display(df.describe())

apoe_counts = df["APOE_haplotype"].value_counts()

# Calculate percentages
total_count = apoe_counts["count"].sum()
apoe_percentages = apoe_counts.with_columns(
    [(pl.col("count") / total_count * 100).alias("percentage")]
)

print("\nAPOE Haplotype Percentages:")
print(apoe_percentages)

statistic,participant_id,second_phase,session_order,APOE_rs429358,APOE_rs7412,APOE_haplotype,PICALM_rs3851179,age,sex,education,learning_deficits,BMI,allergies,drugs,ibuprofen_intake,thyroid_diseases,hypertension,diabetes,other_diseases,smoking_status,coffee_status,dementia_history_parents,BDI,SES,RPM,EHI,NEO_NEU,NEO_EXT,NEO_OPE,NEO_AGR,NEO_CON,AUDIT,MINI-COPE_1,MINI-COPE_2,MINI-COPE_3,MINI-COPE_4,…,CVLT_5,CVLT_6,CVLT_7,CVLT_8,CVLT_9,CVLT_10,CVLT_11,CVLT_12,CVLT_13,leukocytes,erythrocytes,hemoglobin,hematocrit,MCV,MCH,MCHC,RDW-CV,platelets,PDW,MPV,P-LCR,neutrophils,lymphocytes,monocytes,eosinophils,basophils,neutrophils_%,lymphocytes_%,monocytes_%,eosinophils_%,basophils_%,total_cholesterol,cholesterol_HDL,non-HDL_cholesterol,LDL_cholesterol,triglycerides,HSV_r
str,str,f64,f64,str,str,str,str,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""192""",192.0,70.0,"""192""","""192""","""192""","""192""",192.0,192.0,174.0,"""175""",192.0,73.0,192.0,191.0,191.0,191.0,192.0,189.0,191.0,192.0,192.0,192.0,192.0,192.0,192.0,191.0,191.0,191.0,191.0,191.0,192.0,190.0,188.0,187.0,188.0,…,192.0,192.0,192.0,192.0,192.0,192.0,192.0,191.0,191.0,76.0,76.0,76.0,76.0,76.0,76.0,76.0,76.0,76.0,76.0,76.0,76.0,76.0,76.0,76.0,76.0,76.0,76.0,76.0,76.0,76.0,76.0,76.0,76.0,76.0,76.0,76.0,76.0
"""null_count""","""0""",0.0,122.0,"""0""","""0""","""0""","""0""",0.0,0.0,18.0,"""17""",0.0,119.0,0.0,1.0,1.0,1.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,2.0,4.0,5.0,4.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,116.0,116.0,116.0,116.0,116.0,116.0,116.0,116.0,116.0,116.0,116.0,116.0,116.0,116.0,116.0,116.0,116.0,116.0,116.0,116.0,116.0,116.0,116.0,116.0,116.0,116.0,116.0,116.0
"""mean""",null,0.411458,0.557143,null,null,null,null,55.052083,0.536458,2.741379,null,26.876458,0.232877,0.510417,1.157068,0.209424,0.230366,0.005208,0.185185,0.52356,1.098958,0.34375,8.520833,31.375,52.791667,84.05724,18.167539,26.157068,31.183246,32.298429,30.769634,3.796875,2.371053,2.414894,1.863636,2.125,…,12.75,13.4375,13.416667,13.635417,4.098958,1.135417,0.75,15.376963,0.52356,6.157237,4.869737,14.648684,44.210526,91.039474,30.131579,33.052632,13.147368,255.671053,13.746053,11.064474,33.934211,3.312763,2.048684,0.552237,0.176184,0.048421,53.514474,33.488158,9.121053,2.827632,0.796053,208.286842,57.072368,151.214474,123.967895,136.232895,0.868421
"""std""",null,0.493384,0.50031,null,null,null,null,3.084736,0.499973,0.651198,null,4.633075,0.425589,0.501198,0.921349,0.663416,0.422174,0.072169,0.389479,0.813194,0.537201,0.518316,7.922371,4.521618,4.541644,21.339283,8.864073,7.028247,6.258715,5.65778,7.058352,3.326144,0.559151,0.559977,0.681052,0.640907,…,2.697682,1.975306,2.534233,2.039504,4.499633,1.644409,1.531519,1.063407,1.108921,1.483845,0.444528,1.378839,3.492147,4.051142,1.635784,1.080305,0.964361,50.925145,2.343356,0.980436,8.175044,0.995279,0.669227,0.140405,0.127069,0.02327,7.782317,7.364925,1.920647,1.820154,0.361272,41.788634,17.398694,37.9591,35.399487,59.981067,0.340279
"""min""","""sub-01""",0.0,0.0,"""C/C""","""C/C""","""e2/e2""","""A/A""",50.0,0.0,1.0,"""0""",19.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,33.0,20.0,0.0,6.0,14.0,12.0,14.0,0.0,0.5,0.5,0.0,0.0,…,5.0,6.0,4.0,3.0,0.0,0.0,0.0,10.0,0.0,3.5,3.9,10.8,37.0,83.0,24.0,28.8,11.8,167.0,10.0,9.4,20.6,1.53,1.0,0.3,0.0,0.01,36.3,15.5,5.9,0.0,0.2,117.9,28.1,74.6,54.12,43.6,0.0
"""25%""",null,0.0,0.0,null,null,null,null,52.0,0.0,3.0,null,23.46,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,29.0,50.0,70.0,12.0,21.0,28.0,29.0,26.0,1.0,2.0,2.0,1.5,2.0,…,11.0,12.0,12.0,13.0,1.0,0.0,0.0,15.0,0.0,5.3,4.6,13.8,42.0,89.0,30.0,32.4,12.6,216.0,11.9,10.2,26.9,2.52,1.6,0.46,0.11,0.03,47.8,27.0,8.0,1.9,0.5,176.0,43.3,117.3,95.14,83.4,1.0
"""50%""",null,0.0,1.0,null,null,null,null,55.0,1.0,


APOE Haplotype Percentages:
shape: (7, 3)
┌────────────────┬───────┬────────────┐
│ APOE_haplotype ┆ count ┆ percentage │
│ ---            ┆ ---   ┆ ---        │
│ str            ┆ u32   ┆ f64        │
╞════════════════╪═══════╪════════════╡
│ e4/e4          ┆ 2     ┆ 1.041667   │
│ e2/e4          ┆ 3     ┆ 1.5625     │
│ e3/e2          ┆ 2     ┆ 1.041667   │
│ e3/e2          ┆ 19    ┆ 9.895833   │
│ e3/e3          ┆ 119   ┆ 61.979167  │
│ e2/e2          ┆ 1     ┆ 0.520833   │
│ e3/e4          ┆ 46    ┆ 23.958333  │
└────────────────┴───────┴────────────┘
